In [1]:
# Run this cell first to ensure protobuf is installed before importing TensorFlow
%pip install --upgrade protobuf==3.20.3 --quiet

Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.20.0 requires protobuf>=5.28.0, but you have protobuf 3.20.3 which is incompatible.


In [2]:
%pip install --upgrade numpy --quiet

# After running this cell, please restart the kernel before running the next cell to avoid ImportError issues.

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip uninstall protobuf -y


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install protobuf==6.32.0


  Using cached protobuf-6.32.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached protobuf-6.32.0-cp310-abi3-win_amd64.whl (435 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

print(tf.__version__)


2.20.0


In [5]:
model = load_model('model.h5')

In [6]:
import pickle
import pandas as pd
import numpy as np

In [7]:
with open('onehot_encoder_geo.pickle', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pickle', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pickle', 'rb') as file:
    scaler = pickle.load(file)

In [8]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [9]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


d:\GenAi\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [10]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [13]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
prediction = model.predict(input_scaled)
prediction 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step


array([[0.0273214]], dtype=float32)

In [15]:
print(f"Prediction Probability: {prediction[0][0]:.4f}")

if prediction[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

Prediction Probability: 0.0273
The customer is not likely to churn.
